# Importing the required libraraies

In [1]:
from glob import glob
from time import time
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation
import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize, pos_tag
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction import text
import re
import string
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem.snowball import SnowballStemmer
from pdfminer.high_level import extract_text
from spacy.lang.de.stop_words import STOP_WORDS

## Format Conversion:

In [2]:
files = glob('Folder where articles existe/*.pdf',recursive = True)
i=0
print("converting pdf to text")
t0 = time()
for file in files:
    print(file)
    text = extract_text(file)
    output_file = open(r"Folder where data will be stored after transformation\\"+str(i+10)+".txt", "w", encoding="utf-8")
    output_file.write(text)
    output_file.close()
    print(i+10)
    i=i+1
print("done in %0.3fs." % (time() - t0))

converting pdf to text
done in 0.000s.


## Primary results of topic modeling

In [3]:
n_samples = 2000
n_features = 1000
n_components = 10
n_top_words = 20

def plot_top_words(model, feature_names, n_top_words, title):
    fig, axes = plt.subplots(int(n_components/5), 5, figsize=(30, 15), sharex=True)
    axes = axes.flatten()
    for topic_idx, topic in enumerate(model.components_):
        #argsort : put element in ascendent or desendent order in this case descendent
        top_features_ind = topic.argsort()[: -n_top_words - 1 : -1]
        top_features = [feature_names[i] for i in top_features_ind]
        weights = topic[top_features_ind]
        ax = axes[topic_idx]
        #create a graphe where we can the name of features with their weights
        ax.barh(top_features, weights, height=0.7,color=['pink','black'])
        ax.set_title(f"Topic {topic_idx +1}", fontdict={"fontsize": 30})
        ax.invert_yaxis()
        ax.tick_params(axis="both", which="major", labelsize=20)
        for i in "top right left".split():
            ax.spines[i].set_visible(False)
        fig.suptitle(title, fontsize=40)
    plt.subplots_adjust(top=0.90, bottom=0.05, wspace=0.90, hspace=0.3)
    plt.show()

## Words Removal 

In [4]:
def cleaning_data(text):
    #remove urls
    text = re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', '', text, flags=re.MULTILINE)
    #remove emails
    text = re.sub('\w*\._?\w*\@\w*\.\w*', ' ', text)
    text = re.sub('\w*\-*\w*\@\w*\.\w*', ' ', text)
    text = re.sub('\w*\@\w*\.\w*', ' ', text)
    #put text in lower letters
    text = text.lower()
    # remove text in square brackets
    text = re.sub('\[.*?\]', ' ', text)
    # remove text in parenthesis
    text = re.sub('\(.*?\)', ' ', text)
    #remove special caracteres.
    text = re.sub('[�√ ⃒∑φ©∀𝛿→∈•∗]', ' ', text)
    #remore numbers
    text = re.sub('\d*\s', ' ', text)
    text = re.sub('\s\d*', ' ', text)
    #remove punctuation (in addition special caracters)
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    #remove one letter
    text = re.sub('\s\w\s', ' ', text)
    #remove some words that means nothing
    text = re.sub(r'\bieee\b', ' ', text)
    text = re.sub(r'\bcontents lists available at sciencedirect\b', ' ', text)
    text = re.sub(r'\blimerick\b', ' ', text)
    text = re.sub(r'\bxplore\b', ' ', text)
    text = re.sub(r'\bpp\b', ' ', text)
    return text

## Storage of TXT files to the new location

In [5]:
articles = []
filenames = glob('Folder where data will be stored after transformation/*.txt')

for filename in filenames:
    file = open(filename, "r+", encoding="utf-8")
    articles.append(cleaning_data(cleaning_data(file.read().rstrip())))
    file.close()

## Stemming and Lemmatization

In [6]:
stemmer = SnowballStemmer(language='english')
article_words1=[]
total_words=0
articlesL=[]
for article in articles:
    article_words1=article.split()
    article = ' '.join([WordNetLemmatizer().lemmatize(w) for w in article_words1])
    article_words1=article.split()
    article = ' '.join([stemmer.stem(w) for w in article_words1])
    articlesL.append(article)
    total_words=total_words+len(article_words1)

## The percentage of each topic’s presence in a document. + Application interface: number of occurrences of each word in topic 6.

In [7]:
def topics_occurancy(model, feature_names):
    D=0
    for article in articlesL:
        F=0
        rate=[]
        name=[]
        table=[]
        left_coordinates=[]
        D=D+1
        print("Document number:",D)
        total_number=0
        article_words=article.split()
        for topic_idx, topic in enumerate(model.components_):
            top_features_ind = topic.argsort()[: -n_top_words - 1 : -1]
            top_features = [feature_names[i] for i in top_features_ind]
            i=0
            R=0
            table_words=[]
            word_rate=[]
            left_coordinate=[]
            z=0
            for word in top_features:
                z=z+1
                left_coordinate.append(z)
                k=0
                table_words.append(word)
                total_one_word=0
                for i in range(len(article_words)):
                    if word == article_words[i]:
                        k=k+1
                        total_one_word=total_one_word+1
                word_rate.append(total_one_word)     
                R=R+k
            table.append(R)
            total_number+=R
            plt.figure(figsize =(20, 5))
            plt.bar(left_coordinate,word_rate,tick_label=table_words,width=0.8,color=['yellow','black'])
            plt.xlabel('words')
            plt.ylabel('number of occurrences')
            plt.title(f"topic {topic_idx+1}")
            plt.show()
        S=0
        y=0
        for topic_ind, topic in enumerate(model.components_):
            y=y+1
            left_coordinates.append(y)
            S=S+(100/total_number)*table[topic_ind]
            F=(100/total_number)*table[topic_ind]
            rate.append(F)
            name.append(f"Topic {topic_ind+1}")
        plt.figure(figsize =(topic_ind, 5))
        plt.bar(left_coordinates,rate,tick_label=name,width=0.8,color=['black','blue'])
        plt.xlabel('TOPICS')
        plt.ylabel('percentage')
        plt.title(f"The percentage of each topic’s presence in document {D}")
        plt.show()
       

# TF-IDF word embedding application

In [8]:
stopset = ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', 've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn', "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't"]
print("Extracting tf-idf features for Topic modeling...")
tfidf_vectorizer = TfidfVectorizer(max_features=n_features, stop_words= stopset)
t0 = time()
tfidf = tfidf_vectorizer.fit_transform(articlesL)
print("done in %0.3fs." % (time() - t0))

Extracting tf-idf features for Topic modeling...


ValueError: empty vocabulary; perhaps the documents only contain stop words

## results representation

In [ ]:
print("\n" * 2,"Fitting LDA models with tf features, n_samples=%d and n_features=%d..."% (n_samples, n_features),)

lda = LatentDirichletAllocation(n_components=n_components,max_iter=10,learning_method="online",learning_offset=50.0,random_state=0,)
t0 = time()
lda.fit(tfidf)
tfidf_feature_names = tfidf_vectorizer.get_feature_names_out()
print("done in %0.3fs."% (time() - t0)) 
plot_top_words(lda, tfidf_feature_names, n_top_words, "Primary results of topic modeling.")
topics_occurancy(lda, tfidf_feature_names)